# OMOP-CDM con postgresql
En este documento se recogen las notas mientras se ha creado una base de datos siguiendo el esquema [OMOP-CDM](https://ohdsi.github.io/CommonDataModel/) mediante Postgresql.

La documentación está [aquí](https://www.postgresql.org/docs/16/index.html). Un buen tutorial se puede encontrar [aquí](https://www.tutorialspoint.com/postgresql/postgresql_python.htm).

## 0. Prerequisitos
Primero es necesario instalar postgresql, esto lo hacemos simplemente lanzando


In [ ]:
sudo apt install postgresql-common 
sudo apt install postgresql

Antes de seguir es importante moverse al directorio de trabajo en el que queramos guardar y generar toda la información.

## 0.1 Preparacion del servidor

Para hacer las cosas en local, hemos creado un entorno de conda con todos los paquetes necesarios.

Las ordenes que hemos seguido son:

In [ ]:
conda create -n py_bps_omop
conda activate py_bps_omop
conda install python 
conda install -c conda-forge sqlalchemy pandas postgresql psycopg recordlinkage
conda install -c conda-forge jupyterhub jupyterlab nodejs nb_conda_kernels openjdk
conda deactivate

Con el paquete creado, podemos montar el servidor local. 
Para crear un servidor local puedes mirar [aquí](https://gist.github.com/gwangjinkim/f13bf596fefa7db7d31c22efd1627c7a)

In [ ]:
# == Crear el servidor local
conda activate py_bps_omop
cd 2024-06_OMOP/OMOP_Postgresql/py_bps_omop_Test01/
initdb -D base
pg_ctl -D base -l logfile start
# Si no funciona, hay que matar el postgres de la maquína
ps aux | grep postgres
kill <process>

# == Arrancar el servidor local
conda activate py_bps_omop
cd Documents/2024-06_OMOP/OMOP_Postgresql/py_bps_omop_Test01/
pg_ctl -D base -l logfile start
# Si no funciona, hay que matar el postgres de la maquína
ps aux | grep postgres
kill <process>
pg_ctl -D base -l logfile start

# si el servidor es nuevo, no habrá bases de datos creadas
# Puedes crear una lanzando
createdb <database> -U <user>
# y desde ahí acceder
psql <database> -U <user>
# Desde dentro de la consola psql habrá primero que crear el esquema
CREATE SCHEMA omop_sch
# Y luego lanzar los scripts para generar la base de datos
# => Mira ipynb 'NotasPostgreSQL.ipynb'
# => Busca la carpeta 'Template' para los scripts
# => Mira la carpeta 'Vocabularies' para posibles vocabularios

## 1. Working routine
Los siguientes comandos permiten crear una base de datos con el formato OMOP-CDM.

Primero tenemos que entrar en la consola de `psql` con el superusuario *postgres*.

In [ ]:
sudo -u postgres psql

Tras esto tenemos que:
1. Crear un rol con nuestro nombre y una contraseña. Es importante crear siempre una contraseña porque si no se definirá por defecto como `NULL` y cualquier intento de autenticar el usuario por contraseña [fallará](https://www.postgresql.org/docs/16/sql-createrole.html). Usamos `CREATE USER` en lugar de `CREATE ROLE` porque así [ya estamos dando permisos al rol para hacer login](https://www.postgresql.org/docs/16/role-attributes.html). 
2. Crear una base de datos. 
3. Creamos un esquema para que contenga y organice las [38 tablas](http://ohdsi.github.io/CommonDataModel/cdm54erd.html) del OMOP-CDM.

Para esto lanzamos desde dentro de psql:

*(¡Ojo a los `;` al final, son importantes!)*

In [ ]:
-- Creamos el rol
CREATE USER <user> CREATEDB PASSWORD '<password>';
-- Creamos la base de datos 
CREATE DATABASE omop_db OWNER <user>;

Tras esto podemos salir de la consola de `psql` mediante el comando `\q`.

Con esto hemos creado el usuario y la base de datos.

Para estructurar la base de datos es necesario descargar los scripts [DDL](https://github.com/OHDSI/CommonDataModel/tree/v5.4.0/inst/ddl/5.4/postgresql) de postgresql de la OHDSI. Esto nos descarga los scripts:

* OMOPCDM_postgresql_5.4_ddl.sql
* OMOPCDM_postgresql_5.4_primary_keys.sql
* OMOPCDM_postgresql_5.4_constraints
* OMOPCDM_postgresql_5.4_indices

Estos scripts crearán automáticamante las tablas que albergarán los datos, estableciendo las restricciones y relaciones entre ellas.

Estos scripts no se pueden lanzar directamente porque es necesario sustituir el nombre genérico del esquema que aparece en los scripts por el nombre del esquema que vayamos a usar. A fecha de 11/06/2024 el nombre que aparece en los archivos es *'@cdmDatabaseSchema'*. Para cambiar esta string por *'omop_sch'*, que es el nombre del esquema que usaremos, se puede lanzar en la shell el comando:

In [ ]:
sed -i -e 's/@cdmDatabaseSchema/omop_sch/g' <file>

Esto hay que hacerlo una vez por cada uno de los cuatro archivos o poner los cuatro nombres separados por espacios en la misma orden. 

Para terminar de construir las tablas primero accedemos a la base de datos como el usuario con el que la creamos:

In [ ]:
psql omop_db -U <user>

Desde dentro de `psql` podemos crear el esquema desde el que colgarán las tablas y ejecutar el script *'OMOPCDM_postgresql_5.4_ddl.sql'*. Este script creará las tablas con sus respectivas columnas y tipos de datos. 

Ejecutamos:

In [ ]:
-- Creamos el esquema
CREATE SCHEMA omop_sch
-- Lanzamos el script que creará las tablas
\i OMOPCDM_postgresql_5.4_ddl.sql

Tras todo esto, deberíamos tener una lista una base de datos, *omop_db*, dentro de la cual hay una serie de tablas agrupadas bajo un esquema, *omop_sch*, siguiendo las directrices de OMOP-CDM. 

Con estos pasos ya estaría montada la estructura básica de la base de datos. Idealmente habría que introducir aquí los datos y [luego](https://github.com/OHDSI/CommonDataModel/blob/v5.3.1/PostgreSQL/OMOP%20CDM%20postgresql%20indexes.txt) lanzar el resto de los scripts, cuya función es facilitar el acceso posterior.

## 2. Meter datos en la tabla
### 2.1 Datos de vocabulario
Gran parte del objetivo de usar una base de datos en formato OMOP-CDM es la estandarización del vocabulario. Esto implica codificar muchos de los campos presentes en las tablas que hemos creado previamente, guardando información para rastrear cada concepto necesario para manejar la base de datos. Todo esto está descrito en detalle en la [wiki](https://github.com/OHDSI/Vocabulary-v5.0/wiki) de OHDSI para los vocabularios estandarizados. Aquí nos limitaremos a explicar brevemente como añadir estas tablas de metainformación a la estructura que ya hemos creado en la sección anterior.

In [ ]:
java -Dumls-apikey=xxx -jar cpt4.jar 5

Ojo que hay que estar en el directorio donde se ha descomprimido el archivo zip. Hay que explicar que la opción `-Duml-apikey=` hay que modificarla para incluir la apikey obtenida mediante este el procedimiento descrito [aquí](https://github.com/OHDSI/Vocabulary-v5.0/wiki/General-Structure,-Download-and-Use#availability-license-download). Este proceso puede tardar varias horas, así que paciencia. Puede ser hasta necesario relanzar varias veces el script hasta que finalice correctamente.

Finalmente, estas tablas csv actualizadas se pueden subir a la base de datos que creamos previamente.

In [ ]:
sed -i -e 's/C:\CDMV5VOCAB\/<ruta>/g' <file>

donde tendríamos que cambiar:
*  `<ruta>`     Por la ruta absoluta hasta el directorio donde están nuestros archivos.
*  `<file>`    Por el nombre que le hayamos dado al script de sql.

***(!!!)** ==> Esto ahora mismo no funciona porque la ruta incluye caracteres (`\`)que interfieren con `sed`. Hay que mirarlo.*

#### 2.1.2 Actualización tablas vocabularios

Para cargar los vocabularios en la base de datos existen un script de sql ya preparados [aquí](https://github.com/OHDSI/CommonDataModel/tree/v5.3.1/PostgreSQL/VocabImport). A fecha 12/04/2024 este script incluye la ruta *'C:\CDMV5VOCAB\'* como directorio donde se encuentras las tablas. Es necesario cambiar esta string por la localización real de los archivos. Para ello usamos el comando `sed`:

Para introducir esta información en *omop_db*, es necesario hacer uso del comando `\copy`. Cada uno de los csv que se han generado se pueden subir a su respectiva tabla mediante los siguientes comandos:

In [ ]:
\copy omop_sch.CONCEPT FROM './CONCEPT.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.CONCEPT_CLASS FROM './CONCEPT_CLASS.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.CONCEPT_SYNONYM FROM './CONCEPT_SYNONYM.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.CONCEPT_RELATIONSHIP FROM './CONCEPT_RELATIONSHIP.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.CONCEPT_ANCESTOR FROM './CONCEPT_ANCESTOR.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.VOCABULARY FROM './VOCABULARY.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.DOMAIN FROM './DOMAIN.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.RELATIONSHIP FROM './RELATIONSHIP.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;
\copy omop_sch.DRUG_STRENGTH FROM 'DRUG_STRENGTH.csv' WITH DELIMITER E'\t' CSV HEADER QUOTE E'\b' ;

### 2.2 Datos locales
Under construction ([1](https://www.postgresqltutorial.com/postgresql-tutorial/import-csv-file-into-posgresql-table/)) ([2](https://stackoverflow.com/questions/2987433/how-to-import-csv-file-data-into-a-postgresql-table)) ([3](https://www.psycopg.org/docs/install.html#install-from-source)).

## 3. Preparación de la base de datos para su acceso

Una vez que la base de datos está poblada, podemos prepararla para facilitar su acceso. Para ello lanzamos los otros 3 scripts en el siguiente orden:

1. Lanzamos *'OMOPCDM_postgresql_5.4_primary_keys.sql'*. Esto genera las **primary_keys** (*xpk_*), que son las columnas de cada tabla que identifican unívocamente cada entrada.
2. Lanzamos *'OMOPCDM_postgresql_5.4_constraints'*. Esto enlaza columnas de cada tabla entre ellas. Cada tabla tendrá una **foreign_key** (*fpk_*) por cada **primary_key** de otra tabla que contenga.
3. Lanzamos *'OMOPCDM_postgresql_5.4_indices'*. Esto ordena las tablas en función de sus **primary_keys**.

Para lanzar estos scripts en orden simplemente entramos en la consola psql y lanzamos los comandos:

In [ ]:
\i OMOPCDM_postgresql_5.4_primary_keys.sql
\i OMOPCDM_postgresql_5.4_constraints
\i OMOPCDM_postgresql_5.4_indices

### * Known issues

##### Errores
Ahora mismo el tercer archivo devuelve un error.

    psql:03_OMOPCDM_postgresql_5.4_constraints.sql:353: ERROR: there is no unique constraint matching given keys for referenced table "cohort".

que estamos aún investigando.

* Puede que se deba a que en la tabla *cohort* no existe un *primary_key* como en la otras. Por eso se queja psql de que no hay *unique keys*.

Está por ver que esto sea un problema o no.

## 4. Ejemplo lectura de db
Código para leer base de datos. La idea es crear el *engine* para acceder a la base de datos y usar el inspector para primero sacar los nombres de las tablas y después iterar sobre esas tablas y extraer las columnas.

Nos devolverá un diccionario con los nombres de las tablas comos *keys* y una lista con los nombres de las columnas.

In [ ]:
import os
import pandas as pd

from helpers.functions import get_table_info, get_column_names, gen_initial_json, gen_df_from_json


La idea general es extraer la información de las tablas con 'get_table_info()' y de esas tablas se pueden obtener las columnas con 'get_column_names()'.

El json organizador se general con la función 'gen_initial_json()', que utiliza la función 'get_url_table()' para extraer información complementaria de la wiki de github de OHDSI.

A continuación se muestra un ejemplo de uso:

In [ ]:
# == == Example of use == ==
# Parameters
dialect = 'postgresql+psycopg'
route = 'omop_db'
schema = 'omop_sch'
# Get table metadata
url = f'{dialect}:///{route}'
tables_OMOP = get_table_info(url, schema=schema)
# Prepare the json
testfile = 'test.json'
os.remove(testfile)
gen_initial_json(tables_OMOP, testfile)
# Get only column names
table_name = 'person'
OMOP_columns_person = get_column_names(tables_OMOP, table_name)